In [1]:
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import BertTokenizer, BertModel
import torch.nn.functional as F
import nltk
from sklearn.metrics import roc_auc_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold

In [2]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

In [3]:
# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ki_shari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ki_shari\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ki_shari\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ki_shari\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
# Define the preprocessing functions
def remove_special_chars(text):
    # Remove special characters and punctuation
    clean_text = re.sub(r'[^\w\s]', '', text)
    return clean_text

def preprocess_text(text):
    # Tokenization
    tokens = nltk.word_tokenize(text.lower())

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]

    return lemmatized_tokens

In [5]:
df = pd.read_csv(r'C:\Users\ki_shari\Downloads\DFF.csv', encoding='latin-1')

In [6]:
df.head(5)

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,ORIGINAL_TEXT,normalized_scores,incon
0,1,1,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav...",0.726538,Medium
1,2,1,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...,0.723638,Medium
2,3,1,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...,0.483963,Low
3,4,1,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...,0.726538,Medium
4,5,1,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...,0.726538,Medium


In [7]:
print(df['ORIGINAL_TEXT'])

0        When least you think so, this product will sav...
1        Lithium batteries are something new introduced...
2        I purchased this swing for my baby. She is 6 m...
3        I was looking for an inexpensive desk calcolat...
4        I only use it twice a week and the results are...
                               ...                        
20995    I bought these for work.  I have high arches, ...
20996    Crocs are one of only two brands of shoes that...
20997    I love moccasins  This fit like it was custom ...
20998    I wish these were a little more durable. I got...
20999    I've been looking for a replacement for my bel...
Name: ORIGINAL_TEXT, Length: 21000, dtype: object


In [8]:
df.loc[df["LABEL"] == "__label1__", "LABEL"] = 1
df.loc[df["LABEL"] == "__label2__", "LABEL"] = 0
df['LABEL']=pd.to_numeric(df['LABEL'])

In [9]:
# Apply the preprocessing steps
df['ORIGINAL_TEXT'] = df['ORIGINAL_TEXT'].apply(remove_special_chars)
df['ORIGINAL_TEXT'] = df['ORIGINAL_TEXT'].apply(preprocess_text)

In [10]:
# Split the data into review text and labels
reviews = df['ORIGINAL_TEXT'].values
labels = df['LABEL'].values

# Convert labels to one-hot encoded format
#labels = tf.keras.utils.to_categorical(labels)

In [11]:
print(df['LABEL'])

0        1
1        1
2        1
3        1
4        1
        ..
20995    0
20996    0
20997    0
20998    0
20999    0
Name: LABEL, Length: 21000, dtype: int64


In [12]:
print(df['ORIGINAL_TEXT'])

0        [least, think, product, save, day, keep, aroun...
1        [lithium, batteri, someth, new, introduc, mark...
2        [purchas, swing, babi, 6, month, pretti, much,...
3        [look, inexpens, desk, calcolatur, work, every...
4        [use, twice, week, result, great, use, teeth, ...
                               ...                        
20995    [bought, work, high, arch, use, arch, support,...
20996    [croc, one, two, brand, shoe, foot, day, work,...
20997    [love, moccasin, fit, like, custom, made, mebr...
20998    [wish, littl, durabl, got, caught, bolt, cross...
20999    [ive, look, replac, belov, kso, trek, own, two...
Name: ORIGINAL_TEXT, Length: 21000, dtype: object


In [23]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)

# Pad sequences to a fixed length
max_len = 100
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Load GloVe word embeddings
embedding_dim = 300  # Change this value based on the GloVe embedding you choose
word_index = tokenizer.word_index
num_words = len(word_index) + 1

embedding_matrix = np.zeros((num_words, embedding_dim))
glove_path = 'C:\\Users\\ki_shari\\Downloads\\glove.6B.300d.txt\\glove.6B.300d.txt'  # Provide the path to your GloVe file

with open(glove_path, encoding='utf-8') as f:
    for line in f:
        word, *vector = line.split()
        if word in word_index:
            idx = word_index[word]
            embedding_matrix[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

In [19]:
# Define the CNN model
def create_model():
    model = Sequential()
    model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Update the evaluation function
def evaluate(y_true, y_pred):
    y_pred = np.round(y_pred).astype(int)
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    return accuracy, f1, recall, precision, auc

# Perform 5-fold cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
accuracy_list, f1_list, recall_list, precision_list, auc_list = [], [], [], [], []

for train_index, test_index in skf.split(padded_sequences, labels):
    X_train, X_test = padded_sequences[train_index], padded_sequences[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    model = create_model()
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001)
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[reduce_lr])
    
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).astype(int)
    accuracy, f1, recall, precision, auc = evaluate(y_test, y_pred)
    
    accuracy_list.append(accuracy)
    f1_list.append(f1)
    recall_list.append(recall)
    precision_list.append(precision)
    auc_list.append(auc)

# Calculate average evaluation metrics
avg_accuracy = np.mean(accuracy_list)
avg_f1 = np.mean(f1_list)
avg_recall = np.mean(recall_list)
avg_precision = np.mean(precision_list)
avg_auc = np.mean(auc_list)

# Print the results
print("Accuracy: {:.4f}".format(avg_accuracy))
print("F1 Score: {:.4f}".format(avg_f1))
print("Recall: {:.4f}".format(avg_recall))
print("Precision: {:.4f}".format(avg_precision))
print("AUC: {:.4f}".format(avg_auc))

Epoch 1/10
263/263 [==============================] - 111s 419ms/step - loss: 0.6681 - accuracy: 0.5823 - val_loss: 0.6579 - val_accuracy: 0.6038 - lr: 0.0010
Epoch 2/10
263/263 [==============================] - 113s 430ms/step - loss: 0.5820 - accuracy: 0.6903 - val_loss: 0.6698 - val_accuracy: 0.6157 - lr: 0.0010
Epoch 3/10
263/263 [==============================] - 178s 677ms/step - loss: 0.4269 - accuracy: 0.8067 - val_loss: 0.8157 - val_accuracy: 0.5845 - lr: 0.0010
Epoch 4/10
263/263 [==============================] - 165s 626ms/step - loss: 0.2168 - accuracy: 0.9245 - val_loss: 1.1784 - val_accuracy: 0.5752 - lr: 0.0010
Epoch 5/10
263/263 [==============================] - 171s 649ms/step - loss: 0.0748 - accuracy: 0.9857 - val_loss: 1.0418 - val_accuracy: 0.5952 - lr: 5.0000e-04
Epoch 6/10
263/263 [==============================] - 167s 634ms/step - loss: 0.0234 - accuracy: 0.9992 - val_loss: 1.1303 - val_accuracy: 0.5938 - lr: 5.0000e-04
Epoch 7/10
263/263 [==================

In [26]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate

def create_cnn_model(vocab_size, embedding_dim, maxlen):
    input_text = Input(shape=(maxlen,))
    
    embedding_layer = Embedding(vocab_size, embedding_dim, input_length=maxlen)(input_text)
    conv1 = Conv1D(128, 3, activation='relu')(embedding_layer)
    conv2 = Conv1D(128, 4, activation='relu')(embedding_layer)
    conv3 = Conv1D(128, 5, activation='relu')(embedding_layer)
    
    pooling1 = GlobalMaxPooling1D()(conv1)
    pooling2 = GlobalMaxPooling1D()(conv2)
    pooling3 = GlobalMaxPooling1D()(conv3)
    
    concatenated = Concatenate()([pooling1, pooling2, pooling3])
    
    dense1 = Dense(64, activation='relu')(concatenated)
    dropout1 = Dropout(0.5)(dense1)
    
    output = Dense(1, activation='sigmoid')(dropout1)
    
    model = Model(inputs=input_text, outputs=output)
    return model

In [27]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import numpy as np

# Update the evaluation function
def evaluate(y_true, y_pred):
    y_pred = np.round(y_pred).astype(int)
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    return accuracy, f1, recall, precision, auc

# Perform 5-fold cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
accuracy_list, f1_list, recall_list, precision_list, auc_list = [], [], [], [], []

for train_index, test_index in skf.split(padded_sequences, labels):
    X_train, X_test = padded_sequences[train_index], padded_sequences[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    model = create_cnn_model(num_words, embedding_dim, max_len)
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001)
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[reduce_lr])
    
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).astype(int)
    accuracy, f1, recall, precision, auc = evaluate(y_test, y_pred)
    
    accuracy_list.append(accuracy)
    f1_list.append(f1)
    recall_list.append(recall)
    precision_list.append(precision)
    auc_list.append(auc)

# Calculate average evaluation metrics
avg_accuracy = np.mean(accuracy_list)
avg_f1 = np.mean(f1_list)
avg_recall = np.mean(recall_list)
avg_precision = np.mean(precision_list)
avg_auc = np.mean(auc_list)

# Print the results
print("Accuracy: {:.4f}".format(avg_accuracy))
print("F1 Score: {:.4f}".format(avg_f1))
print("Recall: {:.4f}".format(avg_recall))
print("Precision: {:.4f}".format(avg_precision))
print("AUC: {:.4f}".format(avg_auc))

Epoch 1/10
263/263 [==============================] - 44s 162ms/step - loss: 0.6664 - accuracy: 0.5879 - val_loss: 0.6392 - val_accuracy: 0.6421 - lr: 0.0010
Epoch 2/10
263/263 [==============================] - 42s 161ms/step - loss: 0.5427 - accuracy: 0.7352 - val_loss: 0.6518 - val_accuracy: 0.6321 - lr: 0.0010
Epoch 3/10
263/263 [==============================] - 42s 160ms/step - loss: 0.2755 - accuracy: 0.8931 - val_loss: 0.8493 - val_accuracy: 0.6210 - lr: 0.0010
Epoch 4/10
263/263 [==============================] - 42s 160ms/step - loss: 0.0670 - accuracy: 0.9819 - val_loss: 1.1510 - val_accuracy: 0.6100 - lr: 0.0010
Epoch 5/10
263/263 [==============================] - 42s 161ms/step - loss: 0.0133 - accuracy: 0.9982 - val_loss: 1.3898 - val_accuracy: 0.6214 - lr: 5.0000e-04
Epoch 6/10
263/263 [==============================] - 42s 161ms/step - loss: 0.0054 - accuracy: 0.9999 - val_loss: 1.5487 - val_accuracy: 0.6302 - lr: 5.0000e-04
Epoch 7/10
263/263 [========================